In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
from os import path
from glob import glob
import numpy as np
from pao_tfn_trainer import train_pao_tfn
from se3cnn.point_utils import difference_matrix
from pao_file_utils import parse_pao_file, write_pao_file

In [4]:
# assuming MOLOPT-DZVP as primary basis set
prim_basis_shells = {
    'H': [2, 1, 0], # two s-shells, one p-shell, no d-shells
    'O': [2, 2, 1], # two s-shells, two p-shells, one d-shell
}
pao_basis_size = 4

pao_files = sorted(glob("2H2O_MD/frame_*/2H2O_pao44-1_0.pao"))
training_pao_files = pao_files[:10]

max_epochs=100
net_H = train_pao_tfn(training_pao_files, prim_basis_shells, pao_basis_size, kind_name="H", max_epochs=max_epochs)
net_O = train_pao_tfn(training_pao_files, prim_basis_shells, pao_basis_size, kind_name="O", max_epochs=max_epochs)

Training NN for kind H using 40 samples.
Epoch: 0  Missmatch: 4.769367  Penalty: 77.192826 Loss: 81.962193
Epoch: 10  Missmatch: 3.485024  Penalty: 2.381678 Loss: 5.866702
Epoch: 20  Missmatch: 2.028019  Penalty: 1.844886 Loss: 3.872905
Epoch: 30  Missmatch: 1.575697  Penalty: 1.223659 Loss: 2.799356
Epoch: 40  Missmatch: 1.108197  Penalty: 1.101138 Loss: 2.209336
Epoch: 50  Missmatch: 0.704972  Penalty: 1.943697 Loss: 2.648668
Epoch: 60  Missmatch: 0.451015  Penalty: 1.123234 Loss: 1.574248
Epoch: 70  Missmatch: 0.259537  Penalty: 1.507688 Loss: 1.767225
Epoch: 80  Missmatch: 0.155032  Penalty: 1.213667 Loss: 1.368700
Epoch: 90  Missmatch: 0.113876  Penalty: 1.221088 Loss: 1.334963
Training NN for kind O using 20 samples.
Epoch: 0  Missmatch: 1.203284  Penalty: 38.197143 Loss: 39.400427
Epoch: 10  Missmatch: 0.950978  Penalty: 1.567417 Loss: 2.518395
Epoch: 20  Missmatch: 0.881326  Penalty: 0.369975 Loss: 1.251301
Epoch: 30  Missmatch: 0.837146  Penalty: 0.459576 Loss: 1.296722
Epoch:

In [6]:
for fn in pao_files:
    kinds, atom2kind, coords, xblocks = parse_pao_file(fn)
    kind_onehot = train_dataset.encode_kind(atom2kind)  # TODO: store kind-encoding in the net
    natoms = coords.shape[0]
    xblocks = []
    for iatom in range(natoms):
        rolled_kinds = np.roll(kind_onehot, shift=-iatom, axis=1)
        rolled_coords =  np.roll(coords, shift=-iatom, axis=0)
        rolled_kinds_torch = torch.from_numpy(rolled_kinds[None,...])
        rolled_coords_torch = torch.from_numpy(rolled_coords[None,...])
        diff_M = difference_matrix(rolled_coords_torch)
        if atom2kind[iatom] == "H":
            output_net = net_H(rolled_kinds_torch, diff_M)
            xblock_net = net_H.decode_xblock(output_net[0,:,0])
        elif atom2kind[iatom] == "O":
            output_net = net_O(rolled_kinds_torch, diff_M)
            xblock_net = net_O.decode_xblock(output_net[0,:,0])
        xblocks.append(xblock_net.detach().numpy())

    frame_dir = path.dirname(fn)
    fn_inferred = frame_dir + "/inferred.pao"
    write_pao_file(coords, xblocks, fn_inferred)
    print("Wrote "+fn_inferred)
    ! cd $frame_dir; /opt/cp2k/exe/local/cp2k.ssmp -i 2H2O_pao44_inferred.inp > 2H2O_pao44_inferred.out

NameError: name 'train_dataset' is not defined